In [67]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
import numpy as np
import matplotlib.pyplot as plt


In [68]:
# 데이터 로드
df = pd.read_excel('./feature_preprocessed.xlsx', engine='openpyxl')
df.head()


,Unnamed: 0,author,year,genre,source,date,price,area,base_charcoal,base_paper,...,trend_art_rental,art_PCA1,art_PCA2,art_PCA3,economic_PCA1,economic_PCA2,economic_PCA3,trend_PCA1,trend_PCA2,trend_PCA3
0,0,0,1969,0,6,2022-12-23,70000000,7747.00,0,1,...,0.0,1.638582,2.047634,0.689058,-1.760236,1.176937,-0.018569,-0.782262,-0.309723,-0.356714
1,1,0,1969,0,1,2022-12-20,21000000,9294.88,0,1,...,0.0,0.988034,2.501505,0.297426,-1.760236,1.176937,-0.018569,-0.861588,-0.304873,-0.451838
2,2,0,1969,0,3,2022-12-06,2200000,4080.00,0,0,...,0.0,1.528116,1.297077,0.943519,-1.760236,1.176937,-0.018569,-0.993799,-0.296790,-0.610379
3,3,0,1946,0,5,2022-11-24,1900000,3937.50,0,1,...,0.0,-1.227254,-0.136068,2.377198,-1.760236,1.176937,-0.018569,-0.914473,-0.301640,-0.515255
4,4,0,1965,0,4,2022-11-17,12000000,1984.00,0,1,...,0.0,1.586735,1.425576,1.382720,-1.760236,1.176937,-0.018569,-0.650052,-0.317806,-0.198173


## 평가함수정의

In [69]:
# 평가 함수 정의 (RMSE, MAE, MAPE)
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mae, mape


In [70]:
def evaluate_models(X_train, X_test, y_train, y_test, component):
    models = {
        'Linear': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso()
    }

    best_rmse = float('inf')
    best_mae = float('inf')
    best_mape = float('inf')
    
    best_model_rmse = None
    best_model_mae = None
    best_model_mape = None

    results = []  # 결과를 저장할 리스트

    for name, model in models.items():
        # 모델 학습
        model.fit(X_train, y_train)

        # 예측
        y_pred = model.predict(X_test)

        # RMSE, MAE, MAPE 평가
        rmse, mae, mape = evaluate_model(y_test, y_pred)

        print(f"{name} RMSE: {rmse:.6f}, MAE: {mae:.6f}, MAPE: {mape:.6f}%")

        # 결과를 딕셔너리 형태로 저장
        results.append({
            "Model": name,
            "Component": component,
            "RMSE": rmse,
            "MAE": mae,
            "MAPE": mape
        })

        # 가장 성능이 좋은 모델을 선택 (RMSE 기준)
        if rmse < best_rmse:
            best_rmse = rmse
            best_model_rmse = name

        if mae < best_mae:
            best_mae = mae
            best_model_mae = name

        if mape < best_mape:
            best_mape = mape
            best_model_mape = name

    # MLP 평가 (이 부분은 GridSearch로 최적의 파라미터 찾음)
    param_grid = {
        'hidden_layer_sizes': [(128, 64, 32, 16), (256, 128, 64, 32)],  # 레이어 4개 구성
        'max_iter': [500, 1000],  # 반복 횟수
        'solver': ['adam', 'sgd']  # 옵티마이저
    }

    mlp_model = MLPRegressor(random_state=42)

    grid_search = GridSearchCV(mlp_model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    y_pred_mlp = grid_search.best_estimator_.predict(X_test)

    rmse_mlp, mae_mlp, mape_mlp = evaluate_model(y_test, y_pred_mlp)

    print(f"MLP - RMSE: {rmse_mlp:.6f}, MAE: {mae_mlp:.6f}, MAPE: {mape_mlp:.6f}%")

    # MLP 결과도 저장
    results.append({
        "Model": "MLP",
        "Component": component,
        "RMSE": rmse_mlp,
        "MAE": mae_mlp,
        "MAPE": mape_mlp
    })

    print(f"Best RMSE Model: {best_model_rmse} with RMSE: {best_rmse}")
    print(f"Best MAE Model: {best_model_mae} with MAE: {best_mae}")
    print(f"Best MAPE Model: {best_model_mape} with MAPE: {best_mape}%")

    return results

In [71]:
# PCA 구성 요소를 사용하는 Feature Group들에 대해 평가
def evaluate_feature_group(df, features, y, component, random_state=42):
    X_all = df[features]
    X_train_all, X_test_all, y_train, y_test = train_test_split(X_all, y, test_size=0.2, random_state=random_state)
    result = evaluate_models(X_train_all, X_test_all, y_train, y_test, component)
    return result

# PCA 모델 평가
# 전체 PCA 그룹에 대해 실행
def evaluate_pca_models(df, feature_combinations, y, random_states):
    final_results = []

    for feature_group_name, features in feature_combinations.items():
        print(f"\nEvaluating {feature_group_name}:")

        for state in random_states:
            # 각 PCA Component에 대해 평가
            for idx, component in enumerate(["ALL (PC1 + PC2 + PC3)", "PC1", "PC2", "PC3"]):
                current_features = features if idx == 0 else [features[idx-1]]  # 첫 번째는 ALL, 이후는 각각의 PC만
                result = evaluate_feature_group(df, current_features, y, component, random_state=state)

                # 평가 결과에 Feature Group 및 Random State 정보 추가
                for res in result:
                    res['Feature Group'] = feature_group_name
                    res['Random State'] = state

                final_results.extend(result)

    return final_results

In [72]:

# 실행 코드
feature_combinations = {
    'Art': ['art_PCA1', 'art_PCA2', 'art_PCA3'],
    'Art + Economic': ['art_PCA1', 'art_PCA2', 'art_PCA3', 'economic_PCA1', 'economic_PCA2', 'economic_PCA3'],
    'Art + Trend': ['art_PCA1', 'art_PCA2', 'art_PCA3', 'trend_PCA1', 'trend_PCA2', 'trend_PCA3'],
    'Art + Economic + Trend': ['art_PCA1', 'art_PCA2', 'art_PCA3', 'economic_PCA1', 'economic_PCA2', 'economic_PCA3', 'trend_PCA1', 'trend_PCA2']
}

# 종속 변수 변환
y = np.log1p(df['price'].astype(float))

# 결과 실행
df_results = evaluate_pca_models(df, feature_combinations, y, random_states=[42, 123])

# 결과 출력
df_results_df = pd.DataFrame(df_results)
print(df_results_df)



Evaluating Art:
Linear RMSE: 1.099935, MAE: 0.860922, MAPE: 5.741580%
Ridge RMSE: 1.099932, MAE: 0.860948, MAPE: 5.741681%
Lasso RMSE: 1.404510, MAE: 1.156618, MAPE: 7.643044%
MLP - RMSE: 0.893887, MAE: 0.686334, MAPE: 4.570211%
Best RMSE Model: Ridge with RMSE: 1.0999319085820158
Best MAE Model: Linear with MAE: 0.8609215498797739
Best MAPE Model: Linear with MAPE: 5.741579588812786%
Linear RMSE: 1.626088, MAE: 1.319782, MAPE: 8.721742%
Ridge RMSE: 1.626090, MAE: 1.319780, MAPE: 8.721723%
Lasso RMSE: 1.694628, MAE: 1.367254, MAPE: 9.033148%
MLP - RMSE: 1.429502, MAE: 1.136967, MAPE: 7.491584%
Best RMSE Model: Linear with RMSE: 1.626087645514385
Best MAE Model: Ridge with MAE: 1.3197795390200868
Best MAPE Model: Ridge with MAPE: 8.721723445431406%
Linear RMSE: 1.275953, MAE: 1.020469, MAPE: 6.778897%
Ridge RMSE: 1.275950, MAE: 1.020525, MAPE: 6.779216%
Lasso RMSE: 1.404510, MAE: 1.156618, MAPE: 7.643044%
MLP - RMSE: 1.210409, MAE: 0.946061, MAPE: 6.308967%
Best RMSE Model: Ridge with 

/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP - RMSE: 0.917282, MAE: 0.714094, MAPE: 4.820133%
Best RMSE Model: Ridge with RMSE: 1.056192376802266
Best MAE Model: Linear with MAE: 0.8093175303874152
Best MAPE Model: Linear with MAPE: 5.518534945073139%
Linear RMSE: 1.566972, MAE: 1.279824, MAPE: 8.762749%
Ridge RMSE: 1.566966, MAE: 1.279822, MAPE: 8.762739%
Lasso RMSE: 1.622412, MAE: 1.351661, MAPE: 9.232007%
MLP - RMSE: 1.349379, MAE: 1.064427, MAPE: 7.306432%
Best RMSE Model: Ridge with RMSE: 1.5669656262224363
Best MAE Model: Ridge with MAE: 1.2798224474271767
Best MAPE Model: Ridge with MAPE: 8.762739051057965%
Linear RMSE: 1.240456, MAE: 0.978962, MAPE: 6.632023%
Ridge RMSE: 1.240394, MAE: 0.978952, MAPE: 6.631906%
Lasso RMSE: 1.322422, MAE: 1.099676, MAPE: 7.467601%
MLP - RMSE: 1.160100, MAE: 0.906545, MAPE: 6.181549%
Best RMSE Model: Ridge with RMSE: 1.2403942460857427
Best MAE Model: Ridge with MAE: 0.9789518261468135
Best MAPE Model: Ridge with MAPE: 6.631906145407522%
Linear RMSE: 1.595484, MAE: 1.319935, MAPE: 9.066

/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP - RMSE: 0.892296, MAE: 0.685526, MAPE: 4.561299%
Best RMSE Model: Ridge with RMSE: 1.0994466180120757
Best MAE Model: Linear with MAE: 0.8590132964569949
Best MAPE Model: Linear with MAPE: 5.72909661712829%
Linear RMSE: 1.626088, MAE: 1.319782, MAPE: 8.721742%
Ridge RMSE: 1.626090, MAE: 1.319780, MAPE: 8.721723%
Lasso RMSE: 1.694628, MAE: 1.367254, MAPE: 9.033148%
MLP - RMSE: 1.429502, MAE: 1.136967, MAPE: 7.491584%
Best RMSE Model: Linear with RMSE: 1.626087645514385
Best MAE Model: Ridge with MAE: 1.3197795390200868
Best MAPE Model: Ridge with MAPE: 8.721723445431406%
Linear RMSE: 1.275953, MAE: 1.020469, MAPE: 6.778897%
Ridge RMSE: 1.275950, MAE: 1.020525, MAPE: 6.779216%
Lasso RMSE: 1.404510, MAE: 1.156618, MAPE: 7.643044%
MLP - RMSE: 1.210409, MAE: 0.946061, MAPE: 6.308967%
Best RMSE Model: Ridge with RMSE: 1.2759501725502305
Best MAE Model: Linear with MAE: 1.020469421141986
Best MAPE Model: Linear with MAPE: 6.778896615972578%
Linear RMSE: 1.618421, MAE: 1.312336, MAPE: 8.70

...

MLP - RMSE: 0.855441, MAE: 0.684771, MAPE: 4.690752%
Best RMSE Model: Ridge with RMSE: 1.049215088724022
Best MAE Model: Linear with MAE: 0.8067517384921369
Best MAPE Model: Linear with MAPE: 5.506365949624533%
Linear RMSE: 1.566972, MAE: 1.279824, MAPE: 8.762749%
Ridge RMSE: 1.566966, MAE: 1.279822, MAPE: 8.762739%
Lasso RMSE: 1.622412, MAE: 1.351661, MAPE: 9.232007%
MLP - RMSE: 1.349379, MAE: 1.064427, MAPE: 7.306432%
Best RMSE Model: Ridge with RMSE: 1.5669656262224363
Best MAE Model: Ridge with MAE: 1.2798224474271767
Best MAPE Model: Ridge with MAPE: 8.762739051057965%
Linear RMSE: 1.240456, MAE: 0.978962, MAPE: 6.632023%
Ridge RMSE: 1.240394, MAE: 0.978952, MAPE: 6.631906%
Lasso RMSE: 1.322422, MAE: 1.099676, MAPE: 7.467601%
MLP - RMSE: 1.160100, MAE: 0.906545, MAPE: 6.181549%
Best RMSE Model: Ridge with RMSE: 1.2403942460857427
Best MAE Model: Ridge with MAE: 0.9789518261468135
Best MAPE Model: Ridge with MAPE: 6.631906145407522%
Linear RMSE: 1.595484, MAE: 1.319935, MAPE: 9.066

/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the opt

MLP - RMSE: 0.954790, MAE: 0.714444, MAPE: 4.750382%
Best RMSE Model: Ridge with RMSE: 1.1180178757939039
Best MAE Model: Linear with MAE: 0.8701390325074664
Best MAPE Model: Linear with MAPE: 5.786896855407113%
Linear RMSE: 1.626088, MAE: 1.319782, MAPE: 8.721742%
Ridge RMSE: 1.626090, MAE: 1.319780, MAPE: 8.721723%
Lasso RMSE: 1.694628, MAE: 1.367254, MAPE: 9.033148%
MLP - RMSE: 1.429502, MAE: 1.136967, MAPE: 7.491584%
Best RMSE Model: Linear with RMSE: 1.626087645514385
Best MAE Model: Ridge with MAE: 1.3197795390200868
Best MAPE Model: Ridge with MAPE: 8.721723445431406%
Linear RMSE: 1.275953, MAE: 1.020469, MAPE: 6.778897%
Ridge RMSE: 1.275950, MAE: 1.020525, MAPE: 6.779216%
Lasso RMSE: 1.404510, MAE: 1.156618, MAPE: 7.643044%
MLP - RMSE: 1.210409, MAE: 0.946061, MAPE: 6.308967%
Best RMSE Model: Ridge with RMSE: 1.2759501725502305
Best MAE Model: Linear with MAE: 1.020469421141986
Best MAPE Model: Linear with MAPE: 6.778896615972578%
Linear RMSE: 1.618421, MAE: 1.312336, MAPE: 8.7

/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/julianajang/miniforge3/envs/artprice/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the opti

MLP - RMSE: 0.919929, MAE: 0.731159, MAPE: 4.988272%
Best RMSE Model: Ridge with RMSE: 1.0550512841280435
Best MAE Model: Linear with MAE: 0.8076974491636284
Best MAPE Model: Linear with MAPE: 5.50889883505698%
Linear RMSE: 1.566972, MAE: 1.279824, MAPE: 8.762749%
Ridge RMSE: 1.566966, MAE: 1.279822, MAPE: 8.762739%
Lasso RMSE: 1.622412, MAE: 1.351661, MAPE: 9.232007%
MLP - RMSE: 1.349379, MAE: 1.064427, MAPE: 7.306432%
Best RMSE Model: Ridge with RMSE: 1.5669656262224363
Best MAE Model: Ridge with MAE: 1.2798224474271767
Best MAPE Model: Ridge with MAPE: 8.762739051057965%
Linear RMSE: 1.240456, MAE: 0.978962, MAPE: 6.632023%
Ridge RMSE: 1.240394, MAE: 0.978952, MAPE: 6.631906%
Lasso RMSE: 1.322422, MAE: 1.099676, MAPE: 7.467601%
MLP - RMSE: 1.160100, MAE: 0.906545, MAPE: 6.181549%
Best RMSE Model: Ridge with RMSE: 1.2403942460857427
Best MAE Model: Ridge with MAE: 0.9789518261468135
Best MAPE Model: Ridge with MAPE: 6.631906145407522%
Linear RMSE: 1.595484, MAE: 1.319935, MAPE: 9.066

In [78]:
# Feature Group과 Component별로 RMSE가 가장 작은 모델을 선택하는 함수
def get_best_model_by_group_and_component(df):
    # Feature Group과 Component별로 RMSE가 가장 작은 모델을 선택
    best_models = df.loc[df.groupby(['Feature Group', 'Component'])['RMSE'].idxmin()]
    return best_models

# 결과 실행
best_models_df = get_best_model_by_group_and_component(df_results_df)

# 가장 좋은 모델 출력
print(best_models_df)


    Model              Component      RMSE       MAE      MAPE  \
19    MLP  ALL (PC1 + PC2 + PC3)  0.831828  0.662217  4.514987   
23    MLP                    PC1  1.349379  1.064427  7.306432   
27    MLP                    PC2  1.160100  0.906545  6.181549   
31    MLP                    PC3  1.465856  1.231676  8.475226   
51    MLP  ALL (PC1 + PC2 + PC3)  0.917282  0.714094  4.820133   
55    MLP                    PC1  1.349379  1.064427  7.306432   
59    MLP                    PC2  1.160100  0.906545  6.181549   
63    MLP                    PC3  1.465856  1.231676  8.475226   
115   MLP  ALL (PC1 + PC2 + PC3)  0.919929  0.731159  4.988272   
119   MLP                    PC1  1.349379  1.064427  7.306432   
123   MLP                    PC2  1.160100  0.906545  6.181549   
127   MLP                    PC3  1.465856  1.231676  8.475226   
83    MLP  ALL (PC1 + PC2 + PC3)  0.855441  0.684771  4.690752   
87    MLP                    PC1  1.349379  1.064427  7.306432   
91    MLP 

In [79]:
best_models_df.to_excel('model_evaluation_results.xlsx', index=False)

In [74]:

# df_results에서 데이터를 가져와 피벗 테이블 생성
pivot_df = df_results.pivot_table(index='Feature Group', columns='Component', values=['RMSE', 'MAE', 'MAPE'])

# 피벗 테이블 출력
print(pivot_df)

# RMSE, MAE, MAPE를 각각 시각화
metrics = ['RMSE', 'MAE', 'MAPE']
for metric in metrics:
    pivot_metric = df_results.pivot(index='Feature Group', columns='Component', values=metric)
    pivot_metric.plot(kind="bar", figsize=(10, 6), title=f"{metric} by Feature Group and Component")
    
    # 그래프 제목과 축 설정
    plt.title(f"{metric} for Feature Groups and Components")
    plt.ylabel(metric)
    plt.xlabel("Feature Group")
    
    # 범례 설정 및 그래프 표시
    plt.xticks(rotation=0)
    plt.legend(title="Component")
    plt.grid(axis='y')
    plt.show()

AttributeError: 'list' object has no attribute 'pivot_table'

## 각 지표별 결과 Visualizing